# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

PROD_FLAG = True
version = "cloud" if PROD_FLAG else "sandbox"
base_url = f"https://{version}.iexapis.com"

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv("sp_500_stocks.csv")
from config import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [9]:
symbol = "AAPL"
api_url = f"{base_url}/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2524982586604,
 'week52high': 160.54,
 'week52low': 108.27,
 'week52highSplitAdjustOnly': 159.24,
 'week52lowSplitAdjustOnly': 108.99,
 'week52change': 0.3534987935394959,
 'sharesOutstanding': 17078379600,
 'float': 0,
 'avg10Volume': 61669854,
 'avg30Volume': 71786605,
 'day200MovingAvg': 139.8,
 'day50MovingAvg': 147.35,
 'employees': 153303,
 'ttmEPS': 5.3,
 'ttmDividendRate': 0.87253341649801,
 'dividendYield': 0.00576579494601093,
 'nextDividendDate': '',
 'exDividendDate': '2021-08-02',
 'nextEarningsDate': '2021-10-24',
 'peRatio': 28.907819604328296,
 'beta': 1.436219473777184,
 'maxChangePercent': 57.5548297832374,
 'year5ChangePercent': 4.596410926319323,
 'year2ChangePercent': 1.4613703804655667,
 'year1ChangePercent': 0.2878900382187037,
 'ytdChangePercent': 0.12928202898522656,
 'month6ChangePercent': 0.11652508036363698,
 'month3ChangePercent': 0.0162401477430897,
 'month1ChangePercent': 0.02477526293080385,
 'day30ChangePercent

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [37]:
data["year1ChangePercent"]

0.3142387832207159

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [4]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [39]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f"{base_url}/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["price"],
                    data[symbol]["stats"]["year1ChangePercent"],
                    "N/A",
                ],
                index=my_columns
            ),
            ignore_index=True
        )
        
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,161.770,0.545579,N/A
1,AAL,19.640,0.651099,N/A
2,AAP,232.220,0.580840,N/A
3,AAPL,156.776,0.307893,N/A
4,ABBV,113.070,0.376246,N/A
...,...,...,...,...
496,YUM,132.610,0.306667,N/A
497,ZBH,149.920,0.081108,N/A
498,ZBRA,535.090,0.798533,N/A
499,ZION,64.468,1.119144,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [40]:
final_dataframe.sort_values("One-Year Price Return", ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,147,DVN,40.240,4.004232,N/A
1,173,FANG,109.720,3.227234,N/A
2,315,MRO,16.678,3.164186,N/A
3,353,OXY,35.330,2.753084,N/A
4,273,LB,82.870,2.327633,N/A
5,159,EOG,96.300,1.838641,N/A
6,111,COTY,8.658,1.784658,N/A
7,297,MCHP,76.580,1.745545,N/A
8,249,IT,331.300,1.688398,N/A
9,109,COP,78.130,1.656227,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [41]:
def portfolio_input():
    portfolio_size = input("Enter the size of your portfolio:")

    try:
        portfolio_size = float(portfolio_size)
    except:
        print("That is not a number!")
    return portfolio_size

position_size = portfolio_input()/final_dataframe.shape[0]

In [42]:
for i in range(final_dataframe.shape[0]):
    final_dataframe.loc[i, "Number of Shares to Buy"] =\
        math.floor(position_size/final_dataframe.loc[i, "Price"])
final_dataframe

/home/m003/.pyenv/versions/3.9.1/envs/stocks_391/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,147,DVN,40.240,4.004232,4970
1,173,FANG,109.720,3.227234,1822
2,315,MRO,16.678,3.164186,11991
3,353,OXY,35.330,2.753084,5660
4,273,LB,82.870,2.327633,2413
5,159,EOG,96.300,1.838641,2076
6,111,COTY,8.658,1.784658,23100
7,297,MCHP,76.580,1.745545,2611
8,249,IT,331.300,1.688398,603
9,109,COP,78.130,1.656227,2559


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [6]:
hqm_columns = [
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "One-Year Price Return",
    "One-Year Return Percentile",
    "Six-Month Price Return",
    "Six-Month Return Percentile",
    "Three-Month Price Return",
    "Three-Month Return Percentile",
    "One-Month Price Return",
    "One-Month Return Percentile",
    "HQM Score",
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_url = f"{base_url}/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}"
    r = requests.get(batch_api_url)
    print(r.status_code)
    data = r.json()
    for symbol in symbol_string.split(","):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["price"],
                    "N/A",
                    data[symbol]["stats"]["year1ChangePercent"],
                    "N/A",
                    data[symbol]["stats"]["month6ChangePercent"],
                    "N/A",
                    data[symbol]["stats"]["month3ChangePercent"],
                    "N/A",
                    data[symbol]["stats"]["month1ChangePercent"],
                    "N/A",
                    "N/A",
                ],
                index=hqm_columns,
            ),
            ignore_index=True
        )

hqm_dataframe
    

402


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [85]:
time_periods = [
    "One-Year",
    "Six-Month",
    "Three-Month",
    "One-Month",
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        percentile_col = f"{time_period} Return Percentile"
        change_col = f"{time_period} Price Return"
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(
            hqm_dataframe[change_col],
            hqm_dataframe.loc[row, change_col]
        )/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,162.17,N/A,0.529758,0.660679,0.156674,0.694611,0.054794,0.592814,-0.081419,0.043912,N/A
1,AAL,19.37,N/A,0.651417,0.760479,-0.121512,0.115768,-0.125081,0.073852,-0.096349,0.031936,N/A
2,AAP,231.28,N/A,0.575631,0.706587,0.195607,0.770459,0.095357,0.714571,0.078773,0.766467,N/A
3,AAPL,151.98,N/A,0.319119,0.429142,0.116985,0.610778,0.003851,0.383234,0.016874,0.447106,N/A
4,ABBV,110.50,N/A,0.367966,0.493014,0.006121,0.363273,-0.059325,0.175649,0.035511,0.56487,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,131.77,N/A,0.295707,0.387226,0.104020,0.590818,0.026293,0.459082,0.005277,0.38523,N/A
497,ZBH,151.33,N/A,0.082314,0.163673,-0.145099,0.081836,-0.060747,0.165669,-0.007842,0.305389,N/A
498,ZBRA,542.57,N/A,0.816585,0.846307,0.054840,0.493014,-0.037675,0.229541,-0.062193,0.07984,N/A
499,ZION,65.16,N/A,1.118338,0.924152,0.217116,0.818363,0.286080,0.964072,0.086880,0.806387,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [86]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f"{time_period} Return Percentile"])
    hqm_dataframe.loc[row, "HQM Score"] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,162.17,N/A,0.529758,0.660679,0.156674,0.694611,0.054794,0.592814,-0.081419,0.043912,0.498004
1,AAL,19.37,N/A,0.651417,0.760479,-0.121512,0.115768,-0.125081,0.073852,-0.096349,0.031936,0.245509
2,AAP,231.28,N/A,0.575631,0.706587,0.195607,0.770459,0.095357,0.714571,0.078773,0.766467,0.739521
3,AAPL,151.98,N/A,0.319119,0.429142,0.116985,0.610778,0.003851,0.383234,0.016874,0.447106,0.467565
4,ABBV,110.50,N/A,0.367966,0.493014,0.006121,0.363273,-0.059325,0.175649,0.035511,0.56487,0.399202
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,131.77,N/A,0.295707,0.387226,0.104020,0.590818,0.026293,0.459082,0.005277,0.38523,0.455589
497,ZBH,151.33,N/A,0.082314,0.163673,-0.145099,0.081836,-0.060747,0.165669,-0.007842,0.305389,0.179142
498,ZBRA,542.57,N/A,0.816585,0.846307,0.054840,0.493014,-0.037675,0.229541,-0.062193,0.07984,0.412176
499,ZION,65.16,N/A,1.118338,0.924152,0.217116,0.818363,0.286080,0.964072,0.086880,0.806387,0.878244


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [87]:
hqm_dataframe.sort_values("HQM Score", ascending=False, inplace=True)
hqm_dataframe: pd.DataFrame = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DVN,40.000,N/A,3.909393,1.0,0.952964,1.0,0.579454,0.998004,0.257873,0.99002,0.997006
1,MRO,16.400,N/A,3.149882,0.996008,0.582547,0.988024,0.430585,0.994012,0.315737,1.0,0.994511
2,FANG,108.310,N/A,3.181834,0.998004,0.458856,0.98004,0.405330,0.992016,0.260057,0.994012,0.991018
3,OXY,33.500,N/A,2.869043,0.994012,0.437900,0.976048,0.317082,0.974052,0.262082,0.996008,0.98503
4,COP,78.050,N/A,1.635748,0.98004,0.560834,0.986028,0.368668,0.984032,0.185159,0.97006,0.98004
5,EOG,95.250,N/A,1.811262,0.99002,0.374464,0.968064,0.294218,0.968064,0.231470,0.982036,0.977046
6,PXD,189.440,N/A,1.443559,0.97006,0.322325,0.946108,0.346835,0.98004,0.196616,0.974052,0.967565
7,HAL,25.450,N/A,1.107389,0.922156,0.366168,0.966068,0.276510,0.956088,0.257097,0.988024,0.958084
8,SIVB,721.850,N/A,1.624709,0.978044,0.307677,0.928144,0.322290,0.976048,0.153503,0.944112,0.956587
9,OKE,67.340,N/A,1.450669,0.972056,0.333047,0.954092,0.256757,0.94012,0.146765,0.938124,0.951098


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [88]:
position_size = portfolio_input()

In [89]:
position_size = float(position_size)/hqm_dataframe.shape[0]

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, "Number of Shares to Buy"] = (
        position_size/hqm_dataframe.loc[i, "Price"]
    )
hqm_dataframe

/home/m003/.pyenv/versions/3.9.1/envs/stocks_391/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DVN,40.000,500.0,3.909393,1.0,0.952964,1.0,0.579454,0.998004,0.257873,0.99002,0.997006
1,MRO,16.400,1219.512195,3.149882,0.996008,0.582547,0.988024,0.430585,0.994012,0.315737,1.0,0.994511
2,FANG,108.310,184.655156,3.181834,0.998004,0.458856,0.98004,0.405330,0.992016,0.260057,0.994012,0.991018
3,OXY,33.500,597.014925,2.869043,0.994012,0.437900,0.976048,0.317082,0.974052,0.262082,0.996008,0.98503
4,COP,78.050,256.245996,1.635748,0.98004,0.560834,0.986028,0.368668,0.984032,0.185159,0.97006,0.98004
5,EOG,95.250,209.973753,1.811262,0.99002,0.374464,0.968064,0.294218,0.968064,0.231470,0.982036,0.977046
6,PXD,189.440,105.574324,1.443559,0.97006,0.322325,0.946108,0.346835,0.98004,0.196616,0.974052,0.967565
7,HAL,25.450,785.854617,1.107389,0.922156,0.366168,0.966068,0.276510,0.956088,0.257097,0.988024,0.958084
8,SIVB,721.850,27.706587,1.624709,0.978044,0.307677,0.928144,0.322290,0.976048,0.153503,0.944112,0.956587
9,OKE,67.340,297.000297,1.450669,0.972056,0.333047,0.954092,0.256757,0.94012,0.146765,0.938124,0.951098


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [90]:
writer = pd.ExcelWriter("momentum_strategy.xlsx", engine="xlsxwriter")
hqm_dataframe.to_excel(writer, sheet_name="Momentum Strategy", index=False)


## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [91]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [92]:
column_formats = {
    "A": ["Ticker", string_template],
    "B": ["Price", dollar_template],
    "C": ["Number of Shares to Buy", integer_template],
    "D": ["One-Year Price Return", percent_template, percent_template],
    "E": ["One-Year Return Percentile", percent_template],
    "F": ["Six-Month Price Return", percent_template],
    "G": ["Six-Month Return Percentile", percent_template],
    "H": ["Three-Month Price Return", percent_template],
    "I": ["Three-Month Return Percentile", percent_template],
    "J": ["One-Month Price Return", percent_template],
    "K": ["One-Month Return Percentile", percent_template],
    "L": ["HQM Score", percent_template],
}

for column in column_formats.keys():
    writer.sheets["Momentum Strategy"].set_column(
        f"{column}:{column}",
        28,
        column_formats[column][1]
    )
    writer.sheets["Momentum Strategy"].write(
        f"{column}1",
        column_formats[column][0],
        column_formats[column][1]
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [93]:
writer.save()